In [1]:
import pandas as pd
import numpy as np

In [2]:
RANDOM_STATE = 42

results_regression = pd.DataFrame(columns = ['model', 'task', 'R2'])
results_classification = pd.DataFrame(columns = ['model', 'task', 'f1', 'accuracy'])

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston1.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1.Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [4]:
import matplotlib.pyplot as plt

from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression,Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [5]:
# выделим целевую переменную y
X = data.drop('MEDV', axis=1)
y = data['MEDV']

#Ограничиваем размер выборок 
#proc20 = round(X.shape[0] * 0.2)
#y_train = y[:-proc20]
#y_test = y[-proc20:]
#X_train = X[:-proc20]
#X_test = X[-proc20:]

#рандом для однородных выборок
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

y_train.shape, y_test.shape, X_train.shape, X_test.shape


((404,), (102,), (404, 13), (102, 13))

2. Обучите стандартную регрессию, а также Ridge и Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [6]:
### Ваш код ###

r2_lr = 0
r2_ridge = 0
r2_lasso = 0

#Линейная регрессия
#%time
lr = LinearRegression()
lr.fit(X_train, y_train)

r2_lr = lr.score(X_test, y_test)

In [7]:
ridge = Ridge()
ridge.fit(X_train, y_train)
r2_ridge = ridge.score(X_test, y_test)

In [8]:

lasso = Lasso()
lasso.fit(X_train, y_train)
r2_lasso = lasso.score(X_test, y_test)

In [9]:
results_regression.loc[0] = ['LR', 'task2', r2_lr]
results_regression.loc[1] = ['Ridge', 'task2', r2_ridge]
results_regression.loc[2] = ['Lasso', 'task2', r2_lasso]
results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869


3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от 10^-5 до 10^5 (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

In [10]:
### Ваш код ###

r2_ridge_grid_search = 0
r2_ridge_cv = 0
r2_lasso_grid_search = 0
r2_lasso_cv = 0


In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, RidgeCV, Lasso,  LassoCV
import numpy as np

In [12]:
#1) GridSearchCV Ridge
#создаем объект модели
ridge = Ridge()

#Настройка параметров регуляризации
alphas = np.logspace(-5, 5, num=11, endpoint=True, base=10.0)

#создаем объект GridSearchCV с указанием параметров
ridge_grid_search_cv = GridSearchCV(ridge, 
                                    param_grid = {'alpha': alphas}, 
                                    cv=10)
#коэффициент вариации cv

#обучаем модель 
ridge_grid_search_cv.fit(X_train, y_train)

#находим коэффициент детерминации R2
r2_ridge_grid_search = ridge_grid_search_cv.score(X_test, y_test)

#находим коэффициент регуляризации

print('Коэффициент регуляризации', ridge_grid_search_cv.best_params_['alpha'])


Коэффициент регуляризации 1e-05


In [13]:
#2) RidgeCV 
ridge_cv = RidgeCV(alphas=alphas)
#Указание значения атрибута cv вызовет использование перекрестной проверки с помощью GridSearchCV

#обучаем модель 
ridge_cv.fit(X_train, y_train)

#находим коэффициент детерминации R2
r2_ridge_cv = ridge_cv.score(X_test, y_test)

#находим коэффициент регуляризации

print('Коэффициент регуляризации', ridge_cv.alpha_)


Коэффициент регуляризации 0.01


In [14]:
#1) GridSearchCV Lasso
#создаем объект модели
lasso = Lasso()

#Настройка параметров регуляризации
alphas = np.logspace(-5, 5, num=11, endpoint=True, base=10.0)

#создаем объект GridSearchCV с указанием параметров
lasso_grid_search_cv = GridSearchCV(lasso, 
                                    param_grid = {'alpha': alphas}, 
                                    cv=10)

#обучаем модель 
lasso_grid_search_cv.fit(X_train, y_train)

#находим коэффициент детерминации R2
r2_lasso_grid_search = lasso_grid_search_cv.score(X_test, y_test)

#находим коэффициент регуляризации

print('Коэффициент регуляризации', lasso_grid_search_cv.best_params_['alpha'])

Коэффициент регуляризации 1e-05


In [15]:
#2) LassoCV 
lasso_cv = LassoCV(alphas=alphas)
#Указание значения атрибута cv вызовет использование перекрестной проверки с помощью GridSearchCV
# cv=3 по умолчанию

#обучаем модель 
lasso_cv.fit(X_train, y_train)

#находим коэффициент детерминации R2
r2_lasso_cv = lasso_cv.score(X_test, y_test)

#находим коэффициент регуляризации
print('Коэффициент регуляризации', lasso_cv.alpha_)


Коэффициент регуляризации 1e-05


In [16]:
results_regression.loc[3] = ['Ridge_GridSearchCV', 'task3', r2_ridge_grid_search]
results_regression.loc[4] = ['RidgeCV', 'task3', r2_ridge_cv]
results_regression.loc[5] = ['Lasso_GridSearchCV', 'task3', r2_lasso_grid_search]
results_regression.loc[6] = ['LassoCV', 'task3', r2_lasso_cv]
results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668475
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483


4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import r2_score

In [18]:
r2_ridge_standart_scaler = 0
r2_ridge_min_max_scaler = 0
r2_lasso_standart_scaler = 0
r2_lasso_min_max_scaler = 0

In [19]:
#StandardScaler
#cоздаем Pipeline для масштабирования выборки
pipe_ridge = Pipeline(steps = [('scaler', StandardScaler()), ('reg', Ridge())])
pipe_lasso = Pipeline(steps = [('scaler', StandardScaler()), ('reg', Lasso())])

#обучаем модели
pipe_ridge.fit(X_train, y_train)
pipe_lasso.fit(X_train, y_train)

#делаем predict на тестовой выборке
y_predict_ridge = pipe_ridge.predict(X_test)
y_predict_lasso = pipe_lasso.predict(X_test)

#R2 на тестовой выборке
r2_ridge_standart_scaler = r2_score(y_test, y_predict_ridge)
r2_lasso_standart_scaler = r2_score(y_test, y_predict_lasso)


In [20]:
#MinMaxScaler
#cоздаем Pipeline для масштабирования выборки
pipe_ridge = Pipeline(steps = [('scaler', MinMaxScaler()), ('reg', Ridge())])
pipe_lasso = Pipeline(steps = [('scaler', MinMaxScaler()), ('reg', Lasso())])

#обучаем модели
pipe_ridge.fit(X_train, y_train)
pipe_lasso.fit(X_train, y_train)

#делаем predict на тестовой выборке
y_predict_ridge = pipe_ridge.predict(X_test)
y_predict_lasso = pipe_lasso.predict(X_test)

#R2 на тестовой выборке
r2_ridge_min_max_scaler = r2_score(y_test, y_predict_ridge)
r2_lasso_min_max_scaler = r2_score(y_test, y_predict_lasso)

In [21]:
results_regression.loc[7] = ['Ridge_StandardScaler', 'task4', r2_ridge_standart_scaler]
results_regression.loc[8] = ['Ridge_MinMaxScaler', 'task4', r2_ridge_min_max_scaler]
results_regression.loc[9] = ['Lasso_StandardScaler', 'task4', r2_lasso_standart_scaler]
results_regression.loc[10] = ['Lasso_MinMaxScaler', 'task4', r2_lasso_min_max_scaler]
results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668475
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483
7,Ridge_StandardScaler,task4,0.668190
8,Ridge_MinMaxScaler,task4,0.676221
9,Lasso_StandardScaler,task4,0.624045


5.Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [22]:
ridge_cv = RidgeCV(alphas=alphas)
lasso_cv = LassoCV(alphas=alphas)

#StandardScaler
# масштабируем данные
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#обучаем модели на масштабированных данных
ridge_cv.fit(X_train_scaled, y_train)
lasso_cv.fit(X_train_scaled, y_train)

#R2 
r2_ridge_standart_scaler_cv = ridge_cv.score(X_test_scaled, y_test)
r2_lasso_standart_scaler_cv = lasso_cv.score(X_test_scaled, y_test)

# коэффициент регуляризации
print('Коэффициент регуляризации Ridge',  ridge_cv.alpha_)
print('Коэффициент регуляризации Lasso', lasso_cv.alpha_)


Коэффициент регуляризации Ridge 10.0
Коэффициент регуляризации Lasso 1e-05


In [23]:
ridge_cv = RidgeCV(alphas=alphas)
lasso_cv = LassoCV(alphas=alphas)

#MinMaxScaler
# Масштабируем данные
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#обучаем модели на масштабированных данных
ridge_cv.fit(X_train_scaled, y_train)
lasso_cv.fit(X_train_scaled, y_train)

#R2 
r2_ridge_min_max_scaler_cv = ridge_cv.score(X_test_scaled, y_test)
r2_lasso_min_max_scaler_cv = lasso_cv.score(X_test_scaled, y_test)

# коэффициент регуляризации
print('Коэффициент регуляризации Ridge',  ridge_cv.alpha_)
print('Коэффициент регуляризации Lasso',  lasso_cv.alpha_)



Коэффициент регуляризации Ridge 0.1
Коэффициент регуляризации Lasso 1e-05


In [24]:
results_regression.loc[11] = ['Ridge_StandardScaler_CV', 'task5', r2_ridge_standart_scaler_cv]
results_regression.loc[12] = ['Ridge_MinMaxScaler_CV', 'task5', r2_ridge_min_max_scaler_cv]
results_regression.loc[13] = ['Lasso_StandardScaler_CV', 'task5', r2_lasso_standart_scaler_cv]
results_regression.loc[14] = ['Lasso_MinMaxScaler_CV', 'task5', r2_lasso_min_max_scaler_cv]
results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668475
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483
7,Ridge_StandardScaler,task4,0.668190
8,Ridge_MinMaxScaler,task4,0.676221
9,Lasso_StandardScaler,task4,0.624045


6.Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [25]:
from sklearn.preprocessing import PolynomialFeatures




In [26]:
#способ масштабирования StandardScaler
scaler = StandardScaler()

# получаем масштабируемые данные
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

poly = PolynomialFeatures(2)
# Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) 
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

#масштабируем признаки 
X_train_scaled = scaler.fit_transform(X_train_poly)
X_test_scaled = scaler.transform(X_test_poly)

# обучаем модель Ridge с параметрами по умолчанию
ridge = Ridge()
ridge.fit(X_train_scaled, y_train)
r2_ridge_standart_scaler_poly = ridge.score(X_test_scaled, y_test)

# обучаем модель Lasso с параметрами по умолчанию
lasso = Lasso()
lasso.fit(X_train_scaled, y_train)
r2_lasso_standart_scaler_poly = lasso.score(X_test_scaled, y_test)

In [27]:
#способ масштабирования MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

poly = PolynomialFeatures(2)
# Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) 
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

#масштабируем признаки 
X_train_scaled = scaler.fit_transform(X_train_poly)
X_test_scaled = scaler.transform(X_test_poly)

# обучаем модель Ridge с параметрами по умолчанию
ridge = Ridge()
ridge.fit(X_train_scaled, y_train)
r2_ridge_min_max_scaler_poly = ridge.score(X_test_scaled, y_test)


# обучаем модель Lasso с параметрами по умолчанию
lasso = Lasso()
lasso.fit(X_train_scaled, y_train)
r2_lasso_min_max_scaler_poly = lasso.score(X_test_scaled, y_test)




In [28]:
results_regression.loc[15] = ['Ridge_StandardScaler_Poly', 'task6', r2_ridge_standart_scaler_poly]
results_regression.loc[16] = ['Ridge_MinMaxScaler_Poly', 'task6', r2_ridge_min_max_scaler_poly]
results_regression.loc[17] = ['Lasso_StandardScaler_Poly', 'task6', r2_lasso_standart_scaler_poly]
results_regression.loc[18] = ['Lasso_MinMaxScaler_Poly', 'task6', r2_lasso_min_max_scaler_poly]
results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668475
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483
7,Ridge_StandardScaler,task4,0.668190
8,Ridge_MinMaxScaler,task4,0.676221
9,Lasso_StandardScaler,task4,0.624045


7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами

In [29]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.pipeline import make_pipeline

In [30]:
# создаем пайплайн
pipe = make_pipeline(StandardScaler(), PolynomialFeatures())

ridge_cv = RidgeCV(alphas=alphas)
lasso_cv = LassoCV(alphas=alphas)

# получим масштабируем данные
X_train_scaled = pipe.fit_transform(X_train)
X_test_scaled = pipe.transform(X_test)

# обучаем модели на масштабированных данных
ridge_cv.fit(X_train_scaled, y_train)
lasso_cv.fit(X_train_scaled, y_train)

#R2
r2_ridge_standart_scaler_poly_cv = ridge_cv.score(X_test_scaled, y_test)
r2_lasso_standart_scaler_poly_cv = lasso_cv.score(X_test_scaled, y_test)

# Подберите коэффициент регуляризации для Ridge и Lasso
print("Коэффициент регуляризации для Ridge", ridge_cv.alpha_)
print("Коэффициент регуляризации для Lasso", lasso_cv.alpha_)


Коэффициент регуляризации для Ridge 10.0
Коэффициент регуляризации для Lasso 0.1


In [31]:
# создаем пайплайн
pipe = make_pipeline(MinMaxScaler(), PolynomialFeatures())

ridge_cv = RidgeCV(alphas=alphas)
lasso_cv = LassoCV(alphas=alphas)

# получим масштабируем данные
X_train_scaled = pipe.fit_transform(X_train)
X_test_scaled = pipe.transform(X_test)

# обучаем модели на масштабированных данных
ridge_cv.fit(X_train_scaled, y_train)
lasso_cv.fit(X_train_scaled, y_train)

#R2
r2_ridge_min_max_scaler_poly_cv= ridge_cv.score(X_test_scaled, y_test)
r2_lasso_min_max_scaler_poly_cv = lasso_cv.score(X_test_scaled, y_test)

# Подберите коэффициент регуляризации для Ridge и Lasso
print("Коэффициент регуляризации для Ridge", ridge_cv.alpha_)
print("Коэффициент регуляризации для Lasso", lasso_cv.alpha_)

Коэффициент регуляризации для Ridge 0.1
Коэффициент регуляризации для Lasso 0.001


In [32]:
results_regression.loc[19] = ['Ridge_StandardScaler_Poly_CV', 'task7', r2_ridge_standart_scaler_poly_cv]
results_regression.loc[20] = ['Ridge_MinMaxScaler_Poly_CV', 'task7', r2_ridge_min_max_scaler_poly_cv]
results_regression.loc[21] = ['Lasso_StandardScaler_Poly_CV', 'task7', r2_lasso_standart_scaler_poly_cv]
results_regression.loc[22] = ['Lasso_MinMaxScaler_Poly_CV', 'task7', r2_lasso_min_max_scaler_poly_cv]
results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668475
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483
7,Ridge_StandardScaler,task4,0.668190
8,Ridge_MinMaxScaler,task4,0.676221
9,Lasso_StandardScaler,task4,0.624045


8.Подберите наилучшую модель (используйте Pipeline, GridSearchCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [33]:
best_params = {}
print('Параметры лучшей модели:\n', best_params)
r2_best_model = 0
results_regression.loc[23] = ['Best_Model', 'task8', r2_best_model]

Параметры лучшей модели:
 {}


In [34]:

pipe = Pipeline([('scaler', StandardScaler()), ('poly', PolynomialFeatures()), ('reg', Ridge())])

# задаем параметрыдля перебора в GridSearchCV
param_grid = {
    'poly__degree': [1, 2, 3],  # cтепень полинома
    'scaler': [StandardScaler(), MinMaxScaler()],  # метод масштабирования
    'reg': [Ridge(), Lasso()],  # тип регуляризации (L1,L2)
    'reg__alpha': alphas  # коэффициент регуляризации
}

#создаем GridSearchCV с параметрами
grid = GridSearchCV(pipe, param_grid, cv=5)

# обучаем модель 
grid.fit(X_train, y_train)

#автоматически получаем лучшие параметры
best_params= grid.best_params_
print('Параметры лучшей модели:\n', best_params)

# лучший r2 
r2_best_model = grid.best_score_
results_regression.loc[23] = ['Best_Model', 'task8', r2_best_model]
results_regression.loc[23]

Параметры лучшей модели:
 {'poly__degree': 3, 'reg': Lasso(alpha=0.001), 'reg__alpha': 0.001, 'scaler': MinMaxScaler()}


model    Best_Model
task          task8
R2         0.872892
Name: 23, dtype: object

In [35]:
results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668475
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483
7,Ridge_StandardScaler,task4,0.668190
8,Ridge_MinMaxScaler,task4,0.676221
9,Lasso_StandardScaler,task4,0.624045


http://archive.ics.uci.edu/ml/datasets/Adult

In [36]:
data = pd.read_csv('adult.csv')
data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
48838,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

In [37]:
data = pd.read_csv('adult.csv')
# выделим целевую переменную y
X = data.drop('class', axis=1)
y = data['class']

# Замените целевую переменную на числовые значения
y = y.replace({'<=50K': 1, '>50K': 0})
y


0        1
1        1
2        1
3        1
4        1
        ..
48837    1
48838    1
48839    1
48840    1
48841    0
Name: class, Length: 48842, dtype: int64

10.Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score


In [39]:
f1_most_frequent = 0
acc_most_frequent = 0


In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# предсказанное значение class
y_pred =  y_train.value_counts().idxmax() +np.zeros_like(y_test)
# np.zeros_like для преоьразования в фрейм из нулей с формой  исходного фрейма


# метрики accuracy и f1_score
acc_most_frequent = accuracy_score(y_test, y_pred)
f1_most_frequent = f1_score(y_test, y_pred)

results_classification.loc[0] = ['Most Frequent class', 'task10', f1_most_frequent, acc_most_frequent]
results_classification.loc[0]


model       Most Frequent class
task                     task10
f1                     0.862433
accuracy               0.758138
Name: 0, dtype: object

11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [41]:
from sklearn.impute import SimpleImputer

In [42]:
#Выясните, присутствуют ли в данных пропуски.
data.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
class             0
dtype: int64

In [43]:
#Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

imputer = SimpleImputer(strategy='most_frequent')#задаем стратегию замещения пропусков strategy

# Получаем преобразованный датафрейм
data_spacefilled = imputer.fit_transform(data)
data_spacefilled

array([[39, 'State-gov', 77516, ..., 40, 'United-States', '<=50K'],
       [50, 'Self-emp-not-inc', 83311, ..., 13, 'United-States', '<=50K'],
       [38, 'Private', 215646, ..., 40, 'United-States', '<=50K'],
       ...,
       [38, 'Private', 374983, ..., 50, 'United-States', '<=50K'],
       [44, 'Private', 83891, ..., 40, 'United-States', '<=50K'],
       [35, 'Self-emp-inc', 182148, ..., 60, 'United-States', '>50K']],
      dtype=object)

12.Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [44]:

numeric_columns = X.select_dtypes(include='number').columns.values.tolist()
categorical_columns  = X.select_dtypes(include='object').columns.values.tolist()

print("Колонки с числовыми значениями:",numeric_columns)
print("Колонки с категориальными значениями: ", categorical_columns)

Колонки с числовыми значениями: ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
Колонки с категориальными значениями:  ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']


13.Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [45]:
from sklearn.preprocessing import  OneHotEncoder,MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

from sklearn.model_selection import cross_val_score

In [46]:


# Создайте пайплайн по обработке числовых и категориальных значений колонок
numeric_transformer = Pipeline(steps=[('scaler', MinMaxScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Объединяем в один пайплайн
ct = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

# опрещделяем классификаторы по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC

#LogisticRegression, 
lr = LogisticRegression(solver='lbfgs', max_iter=1000)
pipe_lr = Pipeline([
    ('preprocessor', ct),
    ('classifier', lr)
])
f1_LR = cross_val_score(pipe_lr, X, y, cv=5, scoring='f1').mean()
acc_LR = cross_val_score(pipe_lr, X, y, cv=5, scoring='accuracy').mean()


#KNeighborsClassifier
knn = KNeighborsClassifier()
pipe_knn = Pipeline([
    ('preprocessor', ct),
    ('classifier', knn)
])
f1_KNN = cross_val_score(pipe_knn, X, y, cv=5, scoring='f1').mean()
acc_KNN = cross_val_score(pipe_knn, X, y, cv=5, scoring='accuracy').mean()

#LinearSVC
svc = LinearSVC()
pipe_svc = Pipeline([
    ('preprocessor', ct),
    ('classifier', svc)
])
f1_SVM = cross_val_score(pipe_svc, X, y, cv=5, scoring='f1').mean()
acc_SVM = cross_val_score(pipe_svc, X, y, cv=5, scoring='accuracy').mean()                                       
                                           
results_classification.loc[1] = ['LogisticRegression', 'task13', f1_LR, acc_LR]
results_classification.loc[2] = ['KNeighborsClassifier', 'task13', f1_KNN, acc_KNN]
results_classification.loc[3] = ['LinearSVC', 'task13', f1_SVM, acc_SVM]
results_classification

,model,task,f1,accuracy
0,Most Frequent class,task10,0.862433,0.758138
1,LogisticRegression,task13,0.904800,0.850846
2,KNeighborsClassifier,task13,0.886969,0.824741
3,LinearSVC,task13,0.906322,0.852914


14.Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [47]:
# Создайте пайплайн по обработке числовых и категориальных значений колонок
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),# '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). 
    ('scaler', MinMaxScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),# '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). 
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Объединяем в один пайплайн
ct = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

# опрещделяем классификаторы по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC

#LogisticRegression, 
lr = LogisticRegression(solver='lbfgs', max_iter=1000)
pipe_lr = Pipeline([
    ('preprocessor', ct),
    ('classifier', lr)
])
f1_LR = cross_val_score(pipe_lr, X, y, cv=5, scoring='f1').mean()
acc_LR = cross_val_score(pipe_lr, X, y, cv=5, scoring='accuracy').mean()


#KNeighborsClassifier
knn = KNeighborsClassifier()
pipe_knn = Pipeline([
    ('preprocessor', ct),
    ('classifier', knn)
])
f1_KNN = cross_val_score(pipe_knn, X, y, cv=5, scoring='f1').mean()
acc_KNN = cross_val_score(pipe_knn, X, y, cv=5, scoring='accuracy').mean()

#LinearSVC
svc = LinearSVC()
pipe_svc = Pipeline([
    ('preprocessor', ct),
    ('classifier', svc)
])
f1_SVM = cross_val_score(pipe_svc, X, y, cv=5, scoring='f1').mean()
acc_SVM = cross_val_score(pipe_svc, X, y, cv=5, scoring='accuracy').mean()         
                                           
                                           
results_classification.loc[4] = ['LogisticRegression_impute', 'task14', f1_LR, acc_LR]
results_classification.loc[5] = ['KNeighborsClassifier_impute', 'task14', f1_KNN, acc_KNN]
results_classification.loc[6] = ['LinearSVC_impute', 'task14', f1_SVM, acc_SVM]
results_classification

,model,task,f1,accuracy
0,Most Frequent class,task10,0.862433,0.758138
1,LogisticRegression,task13,0.904800,0.850846
2,KNeighborsClassifier,task13,0.886969,0.824741
3,LinearSVC,task13,0.906322,0.852914
4,LogisticRegression_impute,task14,0.904800,0.850846
5,KNeighborsClassifier_impute,task14,0.886969,0.824741
6,LinearSVC_impute,task14,0.906322,0.852914


15.Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [48]:
data = pd.read_csv('adult.csv')

#удаляем значения '?'
data = data.replace('?', np.nan)

# выделим целевую переменную y
X = data.drop('class', axis=1)
y = data['class']


# Замените целевую переменную на числовые значения
y = y.replace({'<=50K': 1, '>50K': 0})


numeric_columns = X.select_dtypes(include='number').columns.values.tolist()
categorical_columns  = X.select_dtypes(include='object').columns.values.tolist()

In [49]:
# Создайте пайплайн по обработке числовых и категориальных значений колонок
numeric_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Объединяем в один пайплайн
ct = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

# опрещделяем классификаторы по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC

#LogisticRegression, 
lr = LogisticRegression(solver='lbfgs', max_iter=1000)
pipe_lr = Pipeline([
    ('preprocessor', ct),
    ('classifier', lr)
])
f1_LR_del_missings = cross_val_score(pipe_lr, X, y, cv=5, scoring='f1').mean()
acc_LR_del_missings = cross_val_score(pipe_lr, X, y, cv=5, scoring='accuracy').mean()


#KNeighborsClassifier
knn = KNeighborsClassifier()
pipe_knn = Pipeline([
    ('preprocessor', ct),
    ('classifier', knn)
])
f1_KNN_del_missings = cross_val_score(pipe_knn, X, y, cv=5, scoring='f1').mean()
acc_KNN_del_missings = cross_val_score(pipe_knn, X, y, cv=5, scoring='accuracy').mean()

#LinearSVC
svc = LinearSVC()
pipe_svc = Pipeline([
    ('preprocessor', ct),
    ('classifier', svc)
])
f1_SVM_del_missings = cross_val_score(pipe_svc, X, y, cv=5, scoring='f1').mean()
acc_SVM_del_missings = cross_val_score(pipe_svc, X, y, cv=5, scoring='accuracy').mean()         
                                           
                                           
results_classification.loc[7] = ['LogisticRegression_delete_missings', 'task15', f1_LR_del_missings, acc_LR_del_missings]
results_classification.loc[8] = ['KNeighborsClassifier_delete_missings', 'task15', f1_KNN_del_missings, acc_KNN_del_missings]
results_classification.loc[9] = ['LinearSVC_delete_missings', 'task15', f1_SVM_del_missings, acc_SVM_del_missings]

In [50]:
results_classification



,model,task,f1,accuracy
0,Most Frequent class,task10,0.862433,0.758138
1,LogisticRegression,task13,0.904800,0.850846
2,KNeighborsClassifier,task13,0.886969,0.824741
3,LinearSVC,task13,0.906322,0.852914
4,LogisticRegression_impute,task14,0.904800,0.850846
5,KNeighborsClassifier_impute,task14,0.886969,0.824741
6,LinearSVC_impute,task14,0.906322,0.852914
7,LogisticRegression_delete_missings,task15,0.904826,0.850887
8,KNeighborsClassifier_delete_missings,task15,0.886969,0.824741
9,LinearSVC_delete_missings,task15,0.906322,0.852914


16.Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [51]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [52]:
data = pd.read_csv('adult.csv')

#удаляем значения '?'
data = data.replace('?', np.nan)

# выделим целевую переменную y
X = data.drop('class', axis=1)
y = data['class']


# Замените целевую переменную на числовые значения
y = y.replace({'<=50K': 1, '>50K': 0})


numeric_columns = X.select_dtypes(include='number').columns.values.tolist()
categorical_columns  = X.select_dtypes(include='object').columns.values.tolist()

# Пайплайн для обработки числовых признаков,  с замененными значениями пропусков на самые частые значения.
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', MinMaxScaler())
])

# Пайплайн для обработки категориальных признаков,  с замененными значениями пропусков на самые частые значения.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Объединяем числовой и категориальный пайплайны
ct = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ])

In [53]:
# опрещделяем классификатор RandomForestClassifier
rfc = RandomForestClassifier()

pipe_rfc = Pipeline([
    ('preprocessor', ct),
    ('classifier', rfc)
])

f1_RF = cross_val_score(pipe_rfc, X, y, cv=5, scoring='f1').mean()
acc_RF = cross_val_score(pipe_rfc, X, y, cv=5, scoring='accuracy').mean()

# опрещделяем классификатор GradientBoostingClassifier
gbc = GradientBoostingClassifier()
pipe_gbc = Pipeline([
    ('preprocessor', ct),
    ('classifier', gbc)
])

f1_GB = cross_val_score(pipe_gbc, X, y, cv=5, scoring='f1').mean()
acc_GB = cross_val_score(pipe_gbc, X, y, cv=5, scoring='accuracy').mean()


results_classification.loc[10] = ['RandomForestClassifier', 'task16', f1_RF, acc_RF]
results_classification.loc[11] = ['GradientBoostingClassifier', 'task16', f1_GB, acc_GB]

In [54]:
results_classification

,model,task,f1,accuracy
0,Most Frequent class,task10,0.862433,0.758138
1,LogisticRegression,task13,0.904800,0.850846
2,KNeighborsClassifier,task13,0.886969,0.824741
3,LinearSVC,task13,0.906322,0.852914
4,LogisticRegression_impute,task14,0.904800,0.850846
5,KNeighborsClassifier_impute,task14,0.886969,0.824741
6,LinearSVC_impute,task14,0.906322,0.852914
7,LogisticRegression_delete_missings,task15,0.904826,0.850887
8,KNeighborsClassifier_delete_missings,task15,0.886969,0.824741
9,LinearSVC_delete_missings,task15,0.906322,0.852914


17.Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [55]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [56]:
data = pd.read_csv('adult.csv')

#удаляем значения '?'
data = data.replace('?', np.nan)

# выделим целевую переменную y
X = data.drop('class', axis=1)
y = data['class']


# Замените целевую переменную на числовые значения
y = y.replace({'<=50K': 1, '>50K': 0})


numeric_columns = X.select_dtypes(include='number').columns.values.tolist()
categorical_columns  = X.select_dtypes(include='object').columns.values.tolist()

# Пайплайн для обработки числовых признаков,  с замененными значениями пропусков на самые частые значения.
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', MinMaxScaler())
])

# Пайплайн для обработки категориальных признаков,  с замененными значениями пропусков на самые частые значения.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer()),
    ('encoder', OneHotEncoder())
])




# Объединяем числовой и категориальный пайплайны
ct = ColumnTransformer(transformers=[
    ('numerical', numeric_transformer, numeric_columns),
    ('categorical', categorical_transformer, categorical_columns),
])


In [58]:
### Ваш код ###


estimator = Pipeline(steps=[
    ('transformer', ct),
    ('model', LogisticRegression()),
])
param_grid = {
    'transformer__categorical__imputer__strategy':['mean', 'median', 'most_frequent', 'constant'],
    'transformer__categorical__encoder': [
        LabelEncoder(),
        OneHotEncoder(handle_unknown='ignore', sparse=False)
    ],
    'transformer__numerical__scaler': [
        StandardScaler(),
        MinMaxScaler()
    ],
    'model': [
        LogisticRegression(),
        KNeighborsClassifier(),
        LinearSVC(),
        RandomForestClassifier(),
        GradientBoostingClassifier()
    ],
}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)
grid = GridSearchCV(estimator=estimator, param_grid=param_grid)

grid.fit(X_train, y_train)

best_params = grid.best_params_
print('Параметры лучшей модели:\n', best_params)

f1_best = f1_score(y_test, grid.predict(X_test))
acc_best = accuracy_score(y_test, grid.predict(X_test))

results_classification.loc[12] = ['Best_Model', 'task17', f1_best, acc_best]



Параметры лучшей модели:
 {'model': GradientBoostingClassifier(), 'transformer__categorical__encoder': OneHotEncoder(handle_unknown='ignore', sparse=False), 'transformer__categorical__imputer__strategy': 'constant', 'transformer__numerical__scaler': MinMaxScaler()}


In [ ]:
results_classification